In [39]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 64)
pd.set_option('display.max_rows', 100)
from sklearn.model_selection import GridSearchCV

In [40]:
homePath = 'game'
trainPath = os.path.join(homePath, 'train_ALL.csv')
trainData = pd.read_csv(trainPath)
testPath = os.path.join(homePath, 'test_ALL.csv')
testData = pd.read_csv(testPath)
submPath = os.path.join(homePath, 'submission.csv')
submData = pd.read_csv(submPath)

In [41]:
trainData['emp_length'].fillna(trainData['emp_length'].median(), inplace=True)
trainData['annual_inc'].fillna(trainData['annual_inc'].median(), inplace=True)
trainData['title'].fillna(trainData['title'].median(), inplace=True)
trainData['pub_rec'].fillna(trainData['pub_rec'].median(), inplace=True)
trainData['revol_util'].fillna(trainData['revol_util'].median(), inplace=True)
trainData['total_acc'].fillna(trainData['total_acc'].median(), inplace=True)
trainData['collections_12_mths_ex_med'].fillna(trainData['collections_12_mths_ex_med'].median(), inplace=True)

In [42]:
testData['emp_length'].fillna(testData['emp_length'].median(), inplace=True)
testData['annual_inc'].fillna(testData['annual_inc'].median(), inplace=True)
testData['title'].fillna(testData['title'].median(), inplace=True)
testData['pub_rec'].fillna(testData['pub_rec'].median(), inplace=True)
testData['revol_util'].fillna(testData['revol_util'].median(), inplace=True)
testData['total_acc'].fillna(testData['total_acc'].median(), inplace=True)
testData['collections_12_mths_ex_med'].fillna(testData['collections_12_mths_ex_med'].median(), inplace=True)
del testData['member_id']

In [43]:
from sklearn.metrics import precision_score, recall_score
def calc_f2(label, predict):
    p = precision_score(label, predict)
    r = recall_score(label, predict)
    f2_score = 5*p*r / (4*p + r)
    return f2_score

In [44]:
# trueData = trainData.loc[trainData['acc_now_delinq'] == 1]
# trainData_copy = trainData.copy()
# delLocal = np.array(np.where(np.array(trainData['acc_now_delinq']) == 1))
# for i in delLocal.tolist():
#     trainData_copy.drop(i, axis=0, inplace=True)
# notBlankIndex = np.where(np.array(trainData_copy['member_id']) > 0)
# trainData_copy = trainData_copy.loc[notBlankIndex].reset_index()

In [45]:
trainLabel = trainData['acc_now_delinq']
del trainData['acc_now_delinq']

In [46]:
trainLabel = list(map(int, trainLabel))

In [47]:
# subTrain = pd.concat([trueData, trainData_copy[:3139]], axis=0, ignore_index=True).reset_index()
# subLabel = subTrain['acc_now_delinq']
subTrain = trainData
subLabel = trainLabel

In [53]:
# del subTrain['member_id']
# del trainData['member_id']
# # del subTrain['acc_now_delinq']
# trainData.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,earliest_cr_line,mths_since_last_record,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,8875.0,8875.0,8875.0,36,18.25,321.97,400,540,1.0,90.0,443557,55000.0,291071,201403,45248,0,524215,331275.0,5579,56880,23.65,1985020,50.000000,1.0,10020.0,60.0,31.0,1,0.00,0.00,6078.11000,6078.11,3124.16,1705.39,0.0,1248.56,224.7408,0.0,45.652684,1.0,886868,167112.315780,-32.431020,742.845442,0.000000,144784.000000,1.104076,0.390972,1.126292,0.736275,-8.332643,18821.255670,77.465036,2.272958,2.602202,638.616125,60.101450,16700.000000,1.603401,-1.144795,0.872841
1,12500.0,12500.0,12475.0,36,8.90,396.92,700,930,1.0,20.0,443557,43000.0,329558,201205,207723,0,524215,1.0,338,9282,10.58,1992070,91.980157,0.0,13202.0,88.0,18.0,1,0.00,0.00,14287.61483,14259.05,12500.00,1787.61,0.0,0.00,0.0000,0.0,41.821962,1.0,886868,274008.533695,-136.302086,1626.923030,-9.647884,122551.893522,-1.922435,-4.268232,-0.548076,-3.280653,-47.701204,-34295.479682,71.375772,0.387501,-3.714946,864.813907,54.964590,12603.076099,0.930222,-3.661237,-7.137984
2,33600.0,33600.0,33600.0,60,18.55,863.31,300,420,25.0,100.0,356117,101300.0,329558,201509,601779,0,524215,331275.0,314,59265,20.00,2001050,71.683884,0.0,21467.0,52.0,27.0,0,32552.24,32552.24,2555.30000,2555.30,1047.76,1507.54,0.0,0.00,0.0000,0.0,45.917049,1.0,886868,148742.329589,18.451757,233.910390,770.000000,106215.000000,1.266193,2.725692,1.069500,2.110409,17.901927,41924.879032,69.954666,1.435041,3.286816,6689.198964,62.386586,41300.000000,1.343599,1.081127,2.358519
3,17000.0,17000.0,17000.0,60,9.71,358.78,600,900,3.0,110.0,443557,47840.0,291071,201308,207723,0,524215,4350.0,3231,56880,4.01,1988090,98.539896,0.0,6585.0,28.9,14.0,0,0.00,0.00,18702.48000,18702.48,17000.00,1702.48,0.0,0.00,0.0000,0.0,43.747776,1.0,886868,214456.130323,-92.137796,1176.832154,102.000000,129416.000000,-0.978370,-4.449777,-0.325003,-2.045646,-15.277672,-35269.817955,62.797042,0.578704,-1.830720,235.135755,23.704306,22800.000000,0.664909,-2.409980,-4.899681
4,14000.0,14000.0,14000.0,60,16.99,347.87,400,540,266.0,60.0,356117,30000.0,329558,201507,6253,0,524215,331275.0,1803,26742,31.85,1999030,84.653259,0.0,4167.0,40.5,21.0,0,13230.24,13230.24,1712.92000,1712.92,769.76,943.16,0.0,0.00,0.0000,0.0,46.823131,1.0,886868,64931.852199,16.551495,230.159411,0.000000,15589.000000,1.250887,2.853539,0.993622,1.926283,15.918851,32266.409930,75.753354,1.397840,3.048916,2673.758207,58.816784,10300.000000,1.097526,0.676771,2.127427


In [54]:
# from sklearn.ensemble import RandomForestClassifier
# # max_depth=13, min_samples_split=50, min_samples_leaf=20, max_features=9,
# rnd_clf = RandomForestClassifier(n_estimators=60,  min_samples_split=50, max_depth=13, max_features=9,  oob_score=True)
# rnd_clf.fit(subTrain, subLabel)
# rnd_pred = rnd_clf.predict(trainData)
# rnd_f2 = calc_f2(trainLabel, rnd_pred)
# rnd_f2

In [55]:
# rnd_pred_prob = rnd_clf.predict_proba(trainData)[:, 1]
# print(metrics.accuracy_score(trainLabel, rnd_pred))
# print(metrics.roc_auc_score(trainLabel, rnd_pred_prob))

In [56]:
# from sklearn.ensemble import BaggingClassifier
# bag_rnd = BaggingClassifier(rnd_clf, n_estimators=10, max_samples=1000, bootstrap=True, n_jobs=-1)
# bag_rnd.fit(subTrain, subLabel)
# rnd_pred = bag_rnd.predict(trainData)
# rnd_f2 = calc_f2(trainLabel, rnd_pred)
# rnd_f2

In [57]:
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn import cross_validation, metrics
# gbdt = GradientBoostingClassifier(learning_rate=0.05, min_samples_split=320, min_samples_leaf=7, max_depth=7, 
#                                  max_features='sqrt', subsample=0.7, random_state=10)
# gbdt.fit(subTrain, subLabel)
# y_pred = gbdt.predict(trainData)
# y_pred_prob = gbdt.predict_proba(trainData)[:, 1]
# print(metrics.accuracy_score(trainLabel, y_pred))
# print(metrics.roc_auc_score(trainLabel, y_pred_prob))
# f2 = calc_f2(trainLabel, y_pred)
# f2

In [58]:
# from xgboost.sklearn import XGBClassifier
# xgb1 = XGBClassifier(learning_rate =0.05,
#                      n_estimators=1000,
#                      max_depth=3,
#                      min_child_weight=1,
#                      gamma=0.1,
#                      subsample=0.8,
#                      colsample_bytree=0.8,
#                      objective= 'binary:logistic',
#                      nthread=4,
#                      reg_alpha=0.001,
#                      reg_lambda=0.001,
#                      scale_pos_weight=1,
#                      seed=27)
# xgb1.fit(subTrain, subLabel)
# xgb1_pred = xgb1.predict(trainData)
# xgb1_pred_prob = xgb1.predict_proba(trainData)[:, 1]
# print(metrics.accuracy_score(trainLabel, xgb1_pred))
# print(metrics.roc_auc_score(trainLabel, xgb1_pred_prob))
# xgb1_f2 = calc_f2(trainLabel, xgb1_pred)
# print(xgb1_f2)

In [59]:
# import lightgbm as lgb
# lgbTrainData = lgb.Dataset(subTrain, subLabel)
# param = {'lambda_l1':[0.5,0.8,1]}


In [60]:
# lgb_clf = lgb.LGBMClassifier(learning_rate=0.1,
#                             boosting_type='gbdt',
#                             objective='binary',
#                             n_estimators=1000,
#                             metric='auc',
#                             max_depth=3,
#                             num_leaves=5,
#                             subsample=0.7,
#                             colsample_bytree=0.7,
#                             min_data_in_leaf=450,
#                             feature_fraction=0.7,
#                             bagging_fraction=0.7,
#                             bagging_freq=6,
#                             lambda_l1=1,
#                             lambda_l2=0.001,
#                             min_gain_to_split=0.265,
#                             verbose=5,
#                             is_unbalance=True,
#                             random_state=10)


In [61]:
# lgb_clf.fit(subTrain, subLabel)

In [62]:
# lgb_clf_pred = lgb_clf.predict(trainData)
# lgb_f2 = calc_f2(trainLabel, lgb_clf_pred)
# lgb_f2

In [63]:
# gsearch2 = GridSearchCV(estimator=lgb_clf, param_grid=param, scoring='roc_auc', n_jobs=4, iid=False, cv=5)
# gsearch2.fit(subTrain, subLabel)
# print(gsearch2.best_params_)
# print(gsearch2.best_score_)

In [64]:
from sklearn.ensemble import VotingClassifier
from xgboost.sklearn import XGBClassifier
xgb = XGBClassifier(learning_rate =0.05,
                     n_estimators=1000,
                     max_depth=3,
                     min_child_weight=1,
                     gamma=0.1,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     objective= 'binary:logistic',
                     nthread=4,
                     reg_alpha=0.001,
                     reg_lambda=0.001,
                     scale_pos_weight=1)

import lightgbm as lgb
lgb_clf = lgb.LGBMClassifier(learning_rate=0.1,
                            boosting_type='gbdt',
                            objective='binary',
                            n_estimators=1000,
                            metric='auc',
                            max_depth=3,
                            num_leaves=5,
                            subsample=0.7,
                            colsample_bytree=0.7,
                            min_data_in_leaf=450,
                            feature_fraction=0.7,
                            bagging_fraction=0.7,
                            bagging_freq=6,
                            lambda_l1=1,
                            lambda_l2=0.001,
                            min_gain_to_split=0.265,
                            verbose=5,
                            is_unbalance=True)

from sklearn.ensemble import GradientBoostingClassifier
gbdt = GradientBoostingClassifier(learning_rate=0.05, min_samples_split=320, min_samples_leaf=7, max_depth=7, 
                                 max_features='sqrt', subsample=0.7, random_state=10)

vot = VotingClassifier(estimators=[('xgb', xgb), ('lgb', lgb_clf)], voting='soft')

In [65]:
vot.fit(subTrain,subLabel)

VotingClassifier(estimators=[('xgb', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.1, learning_rate=0.05,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=1000, n_jobs=1, nthread=4, objective='binary:logistic',
   ...da=0.0, silent=True, subsample=0.7,
        subsample_for_bin=200000, subsample_freq=1, verbose=5))],
         flatten_transform=None, n_jobs=1, voting='soft', weights=None)

In [66]:
vot_pred = vot.predict(testData)


/home/heolis/anaconda3/envs/tensorflow/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [67]:
# from sklearn.ensemble import BaggingClassifier
# bag_rnd = BaggingClassifier(vot, n_estimators=10, max_samples=10000, bootstrap=True, n_jobs=-1)
# bag_rnd.fit(subTrain, subLabel)
# rnd_pred = bag_rnd.predict(trainData)
# rnd_f2 = calc_f2(trainLabel, rnd_pred)
# rnd_f2

In [68]:
submData['acc_now_delinq'] = vot_pred

In [69]:
submData.to_csv(submPath, index=False)